<a href="https://colab.research.google.com/github/SiAm1998/Students-performance-analysis-and-visualization/blob/main/Student_Performance_Analysis_and_Visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

input_file = "/content/students_data.csv"
output_file = "/content/updated_students_data.csv"

def read_csv(file_path):
    try:
        return pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return None

def write_csv(data, file_path):
    try:
        data.to_csv(file_path, index=False)
        print(f"Data successfully written to {file_path}")
    except Exception as e:
        print(f"Error writing to file: {e}")

def analyze_performance(data):
    data['Score'] = pd.to_numeric(data['Score'], errors='coerce')

    print("\n--- Performance Analysis ---")
    avg_score = data.groupby(['Student Name', 'Student ID'])['Score'].mean()
    min_score = data.groupby(['Student Name', 'Student ID'])['Score'].min()
    max_score = data.groupby(['Student Name', 'Student ID'])['Score'].max()

    print("Average Scores per Student and ID:\n", avg_score)
    print("\nMinimum Scores per Student and ID:\n", min_score)
    print("\nMaximum Scores per Student and ID:\n", max_score)

    low_attendance = data[data['Attendance Percentage'] < 75]
    missing_assignments = data[data['Assignment Completion'] == "No"]

    print("\nStudents with Attendance < 75%:\n", low_attendance[['Student Name', 'Student ID', 'Attendance Percentage']])
    print("\nStudents with Missing Assignments:\n", missing_assignments[['Student Name', 'Student ID', 'Assignment Completion']])

    failing_students = data[data['Score'] < 50]
    print("\nStudents Failing Specific Subjects:\n", failing_students[['Student Name', 'Student ID', 'Subject', 'Score']])

def visualize_individual_performance(data, student_id):
    student_data = data[data['Student ID'] == student_id]
    if not student_data.empty:
        student_data.plot(x='Subject', y='Score', kind='line', title=f'Performance Trend for {student_id}', marker='o', color='blue')
        plt.xlabel('Subject')
        plt.ylabel('Score')
        plt.show()
    else:
        print("Student not found.")

def visualize_overall_performance(data):
    avg_scores = data.groupby('Subject')['Score'].mean()
    avg_scores.plot(kind='bar', title='Average Scores by Subject', ylabel='Average Score', xlabel='Subject', color='skyblue')
    plt.show()

    attendance_bins = [0, 50, 75, 100]
    attendance_labels = ['<50%', '50-75%', '>75%']
    attendance_categories = pd.cut(data['Attendance Percentage'], bins=attendance_bins, labels=attendance_labels, include_lowest=True)
    attendance_pie = attendance_categories.value_counts()
    attendance_pie.plot(kind='pie', title='Attendance Percentage Categories', autopct='%1.1f%%', colors=['red', 'orange', 'green'])
    plt.show()

def determine_grade(score):
    if score >= 80:
        return 'A+'
    elif score >= 75:
        return 'A'
    elif score >= 70:
        return 'B+'
    elif score >= 60:
        return 'C'
    else:
        return 'Fail'

def generate_performance_report(data):
    data['Grade'] = data['Score'].apply(determine_grade)
    print("\n--- Student Performance Report ---\n", data[['Student Name', 'Student ID', 'Subject', 'Score', 'Grade']])

if __name__ == "__main__":
    student_data = read_csv(input_file)

    if student_data is not None:
        while True:
            print("\n--- Student's Performance Analyzer ---")
            print("1. Analyze Performance")
            print("2. Generate Performance Report")
            print("3. Visualize Performance")
            print("4. Add New Student")
            print("5. Search for Student Information")
            print("6. Update Student Information")
            print("7. Exit")
            choice = input("Enter the number of the option you are looking for (1-7): ")

            if choice == "1":
                analyze_performance(student_data)

            elif choice == "2":
                generate_performance_report(student_data)

            elif choice == "3":
                print("\nVisualization Options:")
                print("1. Individual Performance")
                print("2. Overall Performance")
                vis_choice = input("Enter your choice: ")

                if vis_choice == "1":
                    student_id = input("Enter Student ID: ")
                    visualize_individual_performance(student_data, student_id)
                elif vis_choice == "2":
                    visualize_overall_performance(student_data)
                else:
                    print("Invalid visualization option.")

            elif choice == "4":
                name = input("Enter Student Name: ")
                student_id = input("Enter Student ID (STU-12345 format): ")
                subject = input("Enter Subject: ")
                score = int(input("Enter Score: "))
                attendance = float(input("Enter Attendance Percentage: "))
                assignment_completion = input("Assignment Completion (Yes/No): ")

                new_student = pd.DataFrame({
                    'Student Name': [name],
                    'Student ID': [student_id],
                    'Subject': [subject],
                    'Score': [score],
                    'Attendance Percentage': [attendance],
                    'Assignment Completion': [assignment_completion]
                })
                student_data = pd.concat([student_data, new_student], ignore_index=True)
                print("New student added successfully.")

            elif choice == "5":
                student_id = input("Enter Student ID: ")
                result = student_data[student_data['Student ID'] == student_id]
                print("\nSearch Result:\n", result[['Student Name', 'Student ID', 'Subject', 'Score', 'Attendance Percentage', 'Assignment Completion']] if not result.empty else "Student not found.")

            elif choice == "6":
                student_id = input("Enter Student ID to update: ")
                idx = student_data[student_data['Student ID'] == student_id].index
                if not idx.empty:
                    new_score = int(input("Enter new Score: "))
                    new_attendance = float(input("Enter new Attendance Percentage: "))
                    student_data.loc[idx, 'Score'] = new_score
                    student_data.loc[idx, 'Attendance Percentage'] = new_attendance
                    print("Student information updated successfully.")
                else:
                    print("Student not found.")

            elif choice == "7":
                write_csv(student_data, output_file)
                print("Thank you for using our service!")
                break

            else:
                print("Invalid choice. Please try again.")
